<a href="https://colab.research.google.com/github/ft-Azad/Language-Modeling/blob/main/LanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Import Libs

In [ ]:
!pip install -q torchmetrics
!pip install -q torchdata==0.6.1
!pip install 'portalocker>=2.0.0'
!pip install torchtext==0.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torchtext
from torchtext.datasets import WikiText2
from torchdata.datapipes.iter import IterableWrapper, Mapper
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

from torch import optim
from torch.nn import functional as F

import tqdm
import torchmetrics as tm

In [ ]:
for lib in [np, torch, torchtext, tqdm]:
  print(lib.__name__, '-->', lib.__version__)

numpy --> 1.25.2
torch --> 2.0.0+cu117
torchtext --> 0.15.1+cpu
tqdm --> 4.66.4


# Utils

In [ ]:
# !git clone https://github.com/ft-Azad/Language-Modeling

In [ ]:
# %cd ..

In [ ]:
# %ls

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def num_trainable_params(model):
  nums = sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6
  return nums

#Dataset

- Loading Data

In [ ]:
# train_iter, valid_iter, test_iter = WikiText2('/content/')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -q "/content/drive/MyDrive/LanguageModel/Data/wikitext-2-v1.zip" -d '/content/'

In [ ]:
with open('/content/wikitext-2/wiki.train.tokens', 'r') as file:
    train_iter = file.read().splitlines()

with open('/content/wikitext-2/wiki.test.tokens', 'r') as file:
    test_iter = file.read().splitlines()

with open('/content/wikitext-2/wiki.valid.tokens', 'r') as file:
    valid_iter = file.read().splitlines()

train_iter = IterableWrapper(train_iter)
test_iter = IterableWrapper(test_iter)
valid_iter = IterableWrapper(valid_iter)

In [ ]:
train_iter_ = iter(train_iter)
train_iter_

<generator object IterableWrapperIterDataPipe.__iter__ at 0x7c9d42c52570>

In [ ]:
next(train_iter_)

' '

- Build Vocabulary

In [ ]:
txt = ['@Azad hi Azad! 1 n2 3 #45', 'how are are you?']
tokenizer = get_tokenizer('basic_english')
[tokenizer(line) for line in txt]
list(map(tokenizer, txt))

[['@azad', 'hi', 'azad', '!', '1', 'n2', '3', '#45'],
 ['how', 'are', 'are', 'you', '?']]

In [ ]:
vocab = build_vocab_from_iterator(map(tokenizer, txt), specials=['<ukn>'], min_freq=1)
vocab.set_default_index(vocab['<ukn>'])
vocab.get_stoi()

{'n2': 11,
 'how': 10,
 'you': 12,
 'hi': 9,
 'azad': 8,
 '@azad': 7,
 '?': 6,
 '3': 5,
 '1': 4,
 '#45': 3,
 '!': 2,
 'are': 1,
 '<ukn>': 0}

In [ ]:
vocab(['hi', 'azad', '<ukn>', 'hello', 'Hi'.lower()])

[9, 8, 0, 0, 9]

In [ ]:
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [ ]:
len(vocab)

28782

In [ ]:
torch.save(vocab, 'vocab.pt')
torch.save(vocab, '/content/drive/MyDrive/LanguageModel/vocab.pt')